In [4]:
# !pip install git+https://github.com/speechbrain/speechbrain.git@develop
# !pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu


In [5]:
from speechbrain.inference.interfaces import foreign_class

classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", 
                           pymodule_file="custom_interface.py", 
                           classname="CustomEncoderWav2vec2Classifier")

INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch custom_interface.py: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch wav2vec2.ckpt: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch wav2vec2.ckpt: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch model.ckpt: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch model.ckpt: Fetching from HuggingFace Hub 'speechbrain/emotion-recognition-wav2vec2-IEMOCAP' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/emotion-r

In [7]:
!yt-dlp -x --audio-format wav fJ9rUzIMcZQ

[youtube] Extracting URL: fJ9rUzIMcZQ
[youtube] fJ9rUzIMcZQ: Downloading webpage
[youtube] fJ9rUzIMcZQ: Downloading ios player API JSON
[youtube] fJ9rUzIMcZQ: Downloading mweb player API JSON
[youtube] fJ9rUzIMcZQ: Downloading player 2b2385a0
[youtube] fJ9rUzIMcZQ: Downloading m3u8 information
[info] fJ9rUzIMcZQ: Downloading 1 format(s): 251
[download] Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].webm has already been downloaded
[download] 100% of    5.68MiB
[ExtractAudio] Destination: Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].wav
Deleting original file Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].webm (pass -k to keep)


In [8]:
from pydub import AudioSegment
import os

def split_wav(input_file, output_folder, segment_length_ms=10000):
    # Load the audio file
    audio = AudioSegment.from_wav(input_file)
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Get the total length of the audio in milliseconds
    total_length_ms = len(audio)
    
    # Split the audio into 30-second segments
    for i, start in enumerate(range(0, total_length_ms, segment_length_ms)):
        end = start + segment_length_ms
        segment = audio[start:end]
        
        # Generate output filename
        output_filename = f"segment_{start//1000}_{end//1000}.wav"
        output_path = os.path.join(output_folder, output_filename)
        
        # Export the segment
        segment.export(output_path, format="wav")
        print(f"Exported: {output_filename}")

# Example usage
input_file = "Queen – Bohemian Rhapsody (Official Video Remastered) [fJ9rUzIMcZQ].wav"
output_folder = "data/borhap"
split_wav(input_file, output_folder)

Exported: segment_0_10.wav
Exported: segment_10_20.wav
Exported: segment_20_30.wav
Exported: segment_30_40.wav
Exported: segment_40_50.wav
Exported: segment_50_60.wav
Exported: segment_60_70.wav
Exported: segment_70_80.wav
Exported: segment_80_90.wav
Exported: segment_90_100.wav
Exported: segment_100_110.wav
Exported: segment_110_120.wav
Exported: segment_120_130.wav
Exported: segment_130_140.wav
Exported: segment_140_150.wav
Exported: segment_150_160.wav
Exported: segment_160_170.wav
Exported: segment_170_180.wav
Exported: segment_180_190.wav
Exported: segment_190_200.wav
Exported: segment_200_210.wav
Exported: segment_210_220.wav
Exported: segment_220_230.wav
Exported: segment_230_240.wav
Exported: segment_240_250.wav
Exported: segment_250_260.wav
Exported: segment_260_270.wav
Exported: segment_270_280.wav
Exported: segment_280_290.wav
Exported: segment_290_300.wav
Exported: segment_300_310.wav
Exported: segment_310_320.wav
Exported: segment_320_330.wav
Exported: segment_330_340.wav


[Songs that change genre](https://www.reddit.com/r/Music/comments/2xql0a/does_anybody_know_any_songs_that_dramatically/)

In [8]:
%%capture
!wget https://www.dropbox.com/s/u8qyvuyie2op286/spk1_snt1.wav

In [10]:
import speechbrain as sb

source = sb.dataio.dataio.read_audio('spk1_snt1.wav').squeeze()
print(source.shape)

RuntimeError: Error loading audio file: failed to open file spk1_snt1.wav

In [11]:
# ensure file exists 
assert os.path.isfile("data/borhap/segment_0_10.wav")

In [12]:
import torchaudio
print(str(torchaudio.list_audio_backends()))

['sox', 'soundfile']


In [16]:
out_prob, score, index, text_lab = classifier.classify_file('data/borhap/segment_150_160.wav')
print(text_lab)

['neu']


In [18]:
file_path = "Sadness 9074234.wav"
out_prob, score, index, text_lab = classifier.classify_file('data/borhap/segment_150_160.wav')
print(text_lab)

['neu']


In [ ]:
import boto3

def list_s3_objects(bucket_name, prefix=''):
    # Create an S3 client
    s3 = boto3.client('s3')
    
    try:
        # List objects in the bucket
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        
        # Check if there are contents
        if 'Contents' in response:
            print(f"Objects in bucket '{bucket_name}' with prefix '{prefix}':")
            for obj in response['Contents']:
                print(f"- {obj['Key']}")
        else:
            print(f"No objects found in bucket '{bucket_name}' with prefix '{prefix}'")
        
        # Handle pagination if there are more than 1000 objects
        while response.get('IsTruncated', False):
            continuation_token = response.get('NextContinuationToken')
            response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=continuation_token)
            for obj in response.get('Contents', []):
                print(f"- {obj['Key']}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
bucket_name = 'your-bucket-name'
prefix = 'path/to/folder/'  # Optional: use to list objects in a specific "folder"

list_s3_objects(bucket_name, prefix)

In [26]:
import boto3

def download_file_from_s3(bucket_name, s3_file_path, local_file_path):
    # Create an S3 client
    s3 = boto3.client('s3')
    
    try:
        # Download the file
        s3.download_file(bucket_name, s3_file_path, local_file_path)
        print(f"File downloaded successfully to {local_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
bucket_name = os.environ.get("S3_BUCKET")
s3_file_path = 'path/to/your/file.txt'
local_file_path = '/path/to/save/file.txt'

download_file_from_s3(bucket_name, s3_file_path, local_file_path)